## Must add

### Add?

* ax.autoscale_view() for patches vs ax.autoscale
* latex text
* ax.axis('off')
* ax.axisbelow
* gridspec
* ax.secondary_xaxis
* finding all fonts
* ax.twinx for sharing axis
* scale axis
* polar projection
* change color cycle `rcparams['axes.prop_cycle'] = cycler(tab20(range(20)))`
* interactive plotting (backends)
* bound vs xlim - bound changes, lim is set
* wide data seaborn

### Learn

* mouseevent
* callbacks
* backends

### Project

* project idea - create dashboard searching for airbnb listings


### Plots


#### Distribution

* boxplot
* violinplot
* hist

#### Other
* plot_date
* step (maybe)
* stem
* pie
* stackplot
* hexbin
* contour

## What makes matplotlib difficult to learn

1. There are two interfaces - object-oriented and pyplot
2. The documentation is thorough but scattered, with no single, simple, straightforward approach to getting started
3. The `plot` method is very convoluted with so many possible positional arguments. It's also misnamed - it's primary purpose is to create line plots. The other data plotting methods are much more explicit
4. The hierarchy between the figure and axes
5. The word 'axes' - it is not related to the x or y axis. It is the actual 'plot'. I pronounce it 'Axeez' when teaching to differentiate it from 'axis'
6. It's difficult to find the possible values for the extra keyword arguments - linestyle, marker, hatch
7. Understanding the "inches" of the figure
8. The multiple coordinate systems - data, axes, figure, figure-inches, display pixels

In [ ]:
import matplotlib
from matplotlib.figure import Figure
from matplotlib.backends import backend_macosx
shell = get_ipython() 
shell.enable_matplotlib()

fig = Figure()
ax = fig.add_subplot()
canvas = backend_macosx.FigureCanvasMac(fig)
manager = backend_macosx.FigureManagerMac(canvas, 1)

In [ ]:
manager.destroy()

## Summary of backends

* Run `matplotlib.use('backend')`
* Default backend is MacOSX
* Inline backend is module://ipykernel.pylab.backend_inline
* When importing pyplot, code is run to hook up ipython display with matplotlib
    * manually do it with `shell = get_ipython(); shell.enable_matplotlib()`
* All figures have a canvas. The canvas is the literal surface where the pixels get drawn. `fig.canvas`
    * `from matplotlib.backends import backend_macosx; backend_macosx.FigureCanvasMac(fig)`
* If there is a window display, that is handled by the figure manager. `fig.canvas.manager`
    * `backend_macosx.FigureManagerMac(canvas, 1)` - can name the window title bar with this object
* Close window with `manager.destroy()`. Must create new manager to show canvas again.
* Update canvas with `canvas.draw()`

## What happens when `%matplotlib inline` is run

1. Method `matplotlib` is run from IPython/core/magics/pylab.py
1. Calls `shell.enable_matplotlib(gui)` from IPython/core/interactiveshell.py
1. This imports from IPython.core import pylabtools as pt and calls
    1. pt.find_gui_and_backend - imports matplotlib
    1. pt.activate_matplotlib(backend) - backend set, imports pyplot
        1. importing pyplot calls ipython.core.pylabtools.activate_matplotlib again!
    1. pt.configure_inline_support(self, backend)
        1. import ipykernel.pylab.backend_inline.InlineBackend

## What happens when `import matplotlib.pyplot` is run

1. switch_backend is called
    1. Line 220 `backend_mod = importlib.import_module(backend_name)`
    1. ipykernel.pylab.backend_inline is imported which runs `_enable_matplotlib_integration`
    1. which imports from IPython.core.pylabtools and calls activate_matplotlib, configure_inline_support
    1. activate_matplotlib calls:
        1. switch_backend again, but importing backend_name doesn't trigger calling of module
    1. configure_inline_support runs:
        1. registers `flush_figures` from `ipykernel.pylab.backend_inline` for post_execute event
        1. select_figure_formats is run once to register function hook for each type of image
        1. `png_formatter = shell.display_formatter.formatters['image/png']`
        1. `png_formatter.for_type(Figure, lambda fig: print_figure(fig, 'png', **kwargs))`
        1. print_figure function is finally called which calls the fig.canvas.print_figure function to place data in a BytesIO stream. It is this data that is returned.
1. In pyplot, install_repl_displayhook is called and registers event post_execute to draw_all figures - draw using the canvas

## What happens when jupyter notebook is launched from command line

* from ipython import start_python - call start_python
* from IPython.terminal.ipapp import launch_new_instance
* Creates notebook.notebookapp.NotebookApp class that calls initialize and then start
* When starting a new notebook, ipykernel.kernelapp.IPKernelApp is created and initialize and start are called
    * initialize calls init_gui_pylab which sets the backend to inline

## Explanation of why image appears for pyplot

* flush_figures in ipykernel.pylab.inline_backend is always called after cell execution
* it displays all of the 'active' figures.
* active figures are created by pyplot with plt.figure or plt.subplots using `_pylab_helpers` Gcf. A fig manager is added to these
* No fig manager is added when created with Figure


## Setting a display formatter

```python
shell = get_ipython()
png_formatter = shell.display_formatter.formatters['image/png']
png_formatter.for_type(Figure, func)
```